<a href="https://colab.research.google.com/github/nbht98/attack_cat_or_dog_classification/blob/master/TEST_CIFAR_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install mia

  Created wheel for mia: filename=mia-0.1.2-cp36-none-any.whl size=11079 sha256=5407e9dbcba0f6f4947ddb5733955bbd724a13d895876f22cf3e2cad7d9b7193
  Stored in directory: /root/.cache/pip/wheels/e8/83/e4/baae7782aa0d2e45af485d25a7994bab3f76428e483252ce82
Successfully built mia


In [0]:
# Import library 
import numpy as np

from absl import app
from absl import flags

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data


In [0]:
NUM_CLASSES = 10
WIDTH = 32
HEIGHT = 32
CHANNELS = 3

TARGET_EPOCHS = 100 # Number of epochs to train target and shadow models
ATTACK_EPOCHS = 100 # Number of epochs to train attack models.
NUM_SHADOWS = 3 # Number of epochs to train attack models.

In [0]:
def target_model_fn():
  """The architecture of the target (victim) model.
  The attack is white-box, hence the attacker is assumed to know this architecture too."""

  model = tf.keras.models.Sequential()

  model.add(
      layers.Conv2D(
          32,
          (3, 3),
          activation="relu",
          padding="same",
          input_shape=(WIDTH, HEIGHT, CHANNELS),
      )
  )
  model.add(layers.Conv2D(32, (3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
  model.add(layers.Conv2D(64, (3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Flatten())

  model.add(layers.Dense(512, activation="relu"))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
  model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model


def attack_model_fn():
  """Attack model that takes target model predictions and predicts membership.
  Following the original paper, this attack model is specific to the class of the input.
  AttachModelBundle creates multiple instances of this model for each class.
  """
  model = tf.keras.models.Sequential()

  model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

  model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
  model.add(layers.Dense(64, activation="relu"))

  model.add(layers.Dense(1, activation="sigmoid"))
  model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
  return model

In [6]:
def get_data():
    """Prepare CIFAR10 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)
(X_train, y_train), (X_test, y_test) = get_data()

170500096/170498071 [==============================] - 4s 0us/step


In [8]:
# Train the target model.
print("Training the target model...")
target_model = target_model_fn()
# target_model.fit(
#     X_train, y_train, epochs=TARGET_EPOCHS, validation_split=0.1, verbose=True
# )
target_model.load_weights("model900.h5")

Training the target model...


In [5]:
def get_data_shadow():
    """Prepare CIFAR100 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)

(attacker_X, attacker_y), (attacker_X_t, attacker_y_t) = get_data_shadow()

169009152/169001437 [==============================] - 4s 0us/step


In [0]:
delete_train = []
delete_test = []
for i in range(len(attacker_y)):
  if(not np.any(attacker_y[i][80:90])):
    delete_train.append(i)

for i in range(len(attacker_y_t)):
  if(not np.any(attacker_y_t[i][80:90])):
    delete_test.append(i)

a_X_train = np.delete(attacker_X, delete_train, 0)
a_y_train = np.delete(attacker_y, delete_train, 0)
a_y_train = np.delete(a_y_train,slice(90,100), 1)
a_y_train = np.delete(a_y_train,slice(0,80), 1)
a_X_test = np.delete(attacker_X_t, delete_test, 0)
a_y_test = np.delete(attacker_y_t, delete_test, 0)
a_y_test = np.delete(a_y_test,slice(90,100), 1)
a_y_test = np.delete(a_y_test,slice(0,80), 1)

In [20]:
SHADOW_DATASET_SIZE_O = 2000

# Train the shadow models.
smb_o = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE_O,
    num_models=NUM_SHADOWS,
)
print("Training the shadow models...")
X_shadow_o, y_shadow_o = smb_o.fit_transform(
    a_X_train,
    a_y_train,
    fit_kwargs=dict(
        epochs=TARGET_EPOCHS,
        verbose=True,
        validation_data=(a_X_test, a_y_test),
    ),
)

Training the shadow models...
Epoch 1/100
63/63 [==============================] - 1s 8ms/step - loss: 2.1021 - accuracy: 0.2010 - val_loss: 1.8624 - val_accuracy: 0.2970
Epoch 2/100
63/63 [==============================] - 0s 6ms/step - loss: 1.8003 - accuracy: 0.3470 - val_loss: 1.6762 - val_accuracy: 0.3890
Epoch 3/100
63/63 [==============================] - 0s 6ms/step - loss: 1.6367 - accuracy: 0.4205 - val_loss: 1.5605 - val_accuracy: 0.4270
Epoch 4/100
63/63 [==============================] - 0s 6ms/step - loss: 1.5175 - accuracy: 0.4600 - val_loss: 1.6642 - val_accuracy: 0.3840
Epoch 5/100
63/63 [==============================] - 0s 6ms/step - loss: 1.4118 - accuracy: 0.5035 - val_loss: 1.3727 - val_accuracy: 0.5170
Epoch 6/100
63/63 [==============================] - 0s 6ms/step - loss: 1.2734 - accuracy: 0.5635 - val_loss: 1.3172 - val_accuracy: 0.5170
Epoch 7/100
63/63 [==============================] - 0s 6ms/step - loss: 1.1838 - accuracy: 0.5730 - val_loss: 1.3424 - val_

In [21]:
# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb_o = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

# Fit the attack models.
print("Training the attack models...")
amb_o.fit(
    X_shadow_o, y_shadow_o, fit_kwargs=dict(epochs=ATTACK_EPOCHS, verbose=True)
)

Training the attack models...
Epoch 1/100
37/37 [==============================] - 0s 2ms/step - loss: 0.6180 - accuracy: 0.7213
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 0.4597 - accuracy: 0.7939
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 0.4031 - accuracy: 0.8142
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3918 - accuracy: 0.8269
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3617 - accuracy: 0.8480
Epoch 6/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3659 - accuracy: 0.8361
Epoch 7/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3603 - accuracy: 0.8514
Epoch 8/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3527 - accuracy: 0.8530
Epoch 9/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3471 - accuracy: 0.8564
Epoch 10/100
37/37 [==============================] - 0s 2ms/step - loss: 0.333

In [22]:
# Test the success of the attack.
ATTACK_TEST_DATASET_SIZE = 4000
# Prepare examples that were in the training, and out of the training.
data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
data_out = X_test[:ATTACK_TEST_DATASET_SIZE], y_test[:ATTACK_TEST_DATASET_SIZE]


# Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

attack_guesses_o = amb_o.predict(attack_test_data)
attack_accuracy_o = np.mean(attack_guesses_o == real_membership_labels)


print(attack_accuracy_o)

0.672125
